In [ ]:
import sys
import os
import pandas as pd
import warnings
import re
import datetime

!mkdir 'RezoJDM-SDS-OpenKE'
!git clone https://github.com/ContentSide/RezoJDM-SDS.git

Cloning into 'RezoJDM-SDS'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 51 (delta 12), reused 48 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [ ]:
triplets_train = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/triplets_train.csv", index_col=False)
triplets_dev = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/triplets_dev.csv", index_col=False)
triplets_test = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/triplets_test.csv", index_col=False)

relation_names = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/relations_ids_names.csv", index_col=False)
nodes_all = pd.read_csv("/content/RezoJDM-SDS/datasets/Filtered_Nodes_Edges/nodes.csv", index_col=False)

In [ ]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

nodes = nodes_in_train.intersection(nodes_in_dev)
nodes = nodes.intersection(nodes_in_test)
nodes = list(nodes)

def remove_delta_nodes(triplets, nodes):
    return triplets[triplets["n1"].isin(nodes) & triplets["n2"].isin(nodes)].reset_index()

triplets_train = remove_delta_nodes(triplets_train, nodes)
triplets_dev = remove_delta_nodes(triplets_dev, nodes)
triplets_test = remove_delta_nodes(triplets_test, nodes)

In [ ]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

nodes = nodes_in_train.union(nodes_in_dev)
nodes = nodes.union(nodes_in_test)
nodes = list(nodes)

In [ ]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

len(nodes_in_train), len(nodes_in_dev), len(nodes_in_test)

(15525, 15523, 15523)

In [ ]:
nodes_name = nodes_all[nodes_all["eid"].isin(nodes)].reset_index()[["eid", "n"]]
len(nodes_name), nodes_all.shape[0]

(15525, 73106)

In [ ]:
nodes_name = nodes_name.rename_axis('new_eid').reset_index()
jdm2oke_node_mapping = {}
for _ , row in nodes_name.iterrows():
    jdm2oke_node_mapping[row.eid] = row.new_eid
len(jdm2oke_node_mapping)

15525

In [ ]:
len(triplets_train["t"].unique()), len(triplets_dev["t"].unique()), len(triplets_test["t"].unique())

(54, 54, 54)

In [ ]:
triplets_train.shape[0], triplets_dev.shape[0], triplets_test.shape[0]

(702470, 88304, 88179)

In [ ]:
relation_names = relation_names[relation_names["relation_id"].isin(triplets_train["t"].unique())].reset_index()[["relation","relation_id"]]
relation_names = relation_names.rename_axis('new_relation_id').reset_index()
jdm2oke_relation_mapping = {}
for _ , row in relation_names.iterrows():
    jdm2oke_relation_mapping[row.relation_id] = row.new_relation_id
len(jdm2oke_relation_mapping)

54

In [ ]:
def create_OpenKE_txt(triplets_csv, file_name):

    line = triplets_csv.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in triplets_csv.iterrows():
        text += (str(jdm2oke_node_mapping[row["n1"]]) +" " +str(jdm2oke_node_mapping[row["n2"]]) + " "+str(jdm2oke_relation_mapping[row["t"]]) +"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()

In [ ]:
create_OpenKE_txt(triplets_train,'./RezoJDM-SDS-OpenKE/train2id.txt')
create_OpenKE_txt(triplets_dev,'./RezoJDM-SDS-OpenKE/valid2id.txt')
create_OpenKE_txt(triplets_test,'./RezoJDM-SDS-OpenKE/test2id.txt')

In [ ]:
def create_OpenKE_ent(nodes_name, file_name):
    
    line = nodes_name.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in nodes_name.iterrows():
        text += (row["n"] +"\t" +str(row["new_eid"])+"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()


def create_OpenKE_rel(nodes_name, file_name):
    
    line = nodes_name.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in nodes_name.iterrows():
        text += (row["relation"] +"\t" +str(row["new_relation_id"])+"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()

In [ ]:
create_OpenKE_ent(nodes_name, "./RezoJDM-SDS-OpenKE/entity2id.txt")
create_OpenKE_rel(relation_names, "./RezoJDM-SDS-OpenKE/relation2id.txt")

In [ ]:
nodes_name

,new_eid,eid,n
0,0,1,vraiment
1,1,6,provenir
2,2,8,traversée
3,3,12,discréditer
4,4,16,Maria Callas
...,...,...,...
15520,15520,437989,Dita von Teese
15521,15521,463696,hémangiome caverneux
15522,15522,463699,chorioangiome
15523,15523,465567,Pierre Boulle


In [ ]:
!zip -r 'RezoJDM-SDS-OpenKE.zip' 'RezoJDM-SDS-OpenKE'

  adding: RezoJDM-SDS-OpenKE/ (stored 0%)
  adding: RezoJDM-SDS-OpenKE/entitiy2id.txt (deflated 52%)
  adding: RezoJDM-SDS-OpenKE/valid2id.txt (deflated 59%)
  adding: RezoJDM-SDS-OpenKE/train2id.txt (deflated 68%)
  adding: RezoJDM-SDS-OpenKE/test2id.txt (deflated 59%)
  adding: RezoJDM-SDS-OpenKE/relation2id.txt (deflated 49%)
